In [12]:
import os
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

# === XML einlesen ===
with open("moduldb-pi2.xml", "r", encoding="ISO-8859-1") as f:
    xml_content = f.read()

soup = BeautifulSoup(xml_content, "xml")
modules = soup.find_all("moduleheader")

# === Module in Dokumente umwandeln ===
docs = []
for module in modules:
    title = module.find("title").text.strip() if module.find("title") else "Kein Titel"
    cid = module.find("cid").text.strip() if module.find("cid") else "Kein Kürzel"
    cp = module.find("cp").text.strip() if module.find("cp") else "?"
    convenor = module.find("convenor").text.strip() if module.find("convenor") else "?"

    # Typen zusammenfassen (V, P, SU ...)
    types = module.find_all("type")
    ctypes = ", ".join(t.text for t in types) if types else "Unbekannt"

    content = f"""
Modul: {title}
Kürzel: {cid}
Leistungspunkte (ECTS): {cp}
Verantwortlich: {convenor}
Veranstaltungstyp(en): {ctypes}
"""
    docs.append(Document(page_content=content.strip()))

# Debug: Beispiel-Inhalt zeigen
print("📄 Beispiel-Inhalt:\n", docs[0].page_content[:500])

# === Dokumente splitten ===
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# === Embeddings & Chroma ===
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# === LLM über Ollama (Server) ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",
    model="llama3-70b",
    temperature=0.5,
    top_p=0.8,
    top_k=10,
    repeat_penalty=1.1,
    presence_penalty=1.2
)

# === Prompt ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Moduldaten:\n"
        "{context}\n\n"
        "Beantworte bitte diese Frage auf einfache Weise:\n"
        "{question}\n\n"
        "Antwort:"
    )
)

# === QA-Kette ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# === Chat starten ===
print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.run(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print(f"⚠️ Fehler: {e}")


📄 Beispiel-Inhalt:
 Modul: Bachelor-Abschlussarbeit
Kürzel: PIB-BT
Leistungspunkte (ECTS): 12
Verantwortlich: Studienleitung
Veranstaltungstyp(en): Unbekannt
🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Die Frage kann nicht beantwortet werden, da es in den gegebenen Moduldaten kein Modul mit dem Namen "Datenbanken" gibt. Es wird nur das Modul "Entwurfsmuster" mit 5 ECTS-Punkten erwähnt.
🤖 Bot: Prof. Dr. Peter Birkner
🤖 Bot: Auf Wiedersehen!
